<a href="https://colab.research.google.com/github/AdarshSRM/FirstSite/blob/main/poringa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests
from bs4 import BeautifulSoup

# The target URL
url = "https://www.poringa.net/posts/imagenes/2011655/Debora-Natalin-Pistarchi---100-Fotos-Hot-Para-Infierno-Rojo.html"

# Headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def scrape_imagebam_links(target_url):
    try:
        print(f"Fetching content from: {target_url}")
        response = requests.get(target_url, headers=headers, timeout=10)
        response.raise_for_status()  # Check for HTTP errors

        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <a> tags (links)
        links = soup.find_all('a', href=True)

        # Filter links that contain 'imagebam.com/image/'
        imagebam_links = []
        for link in links:
            href = link['href']
            if "imagebam.com/image/" in href:
                imagebam_links.append(href)

        # Remove duplicates
        unique_links = list(set(imagebam_links))

        if unique_links:
            # Save to text file
            with open("imagebam_links.txt", "w") as f:
                for link in unique_links:
                    f.write(link + "\n")
            print(f"Success! Found {len(unique_links)} links. Saved to 'imagebam_links.txt'.")
        else:
            print("No ImageBam links found on this page.")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    scrape_imagebam_links(url)

Fetching content from: https://www.poringa.net/posts/imagenes/2011655/Debora-Natalin-Pistarchi---100-Fotos-Hot-Para-Infierno-Rojo.html
Success! Found 100 links. Saved to 'imagebam_links.txt'.


In [8]:
import requests
from bs4 import BeautifulSoup
import os
import time

# Create folder
os.makedirs("downloads", exist_ok=True)

def resolve_direct_link(session, url):
    """
    Attempts to bypass the 'Continue' page to find the images3.imagebam.com link
    """
    # Step 1: Hit the page to get the session/XSRF tokens
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Referer": "https://www.poringa.net/"
    }

    # First request sets the stage
    first_pass = session.get(url, headers=headers)

    # Step 2: Request the same URL with the Referer set to itself.
    # On ImageBam, this usually bypasses the timer/interstitial.
    headers["Referer"] = url
    second_pass = session.get(url, headers=headers)

    soup = BeautifulSoup(second_pass.text, 'html.parser')

    # Look for the 'main-image' or any img tag containing 'imagebam.com/download'
    img = soup.find('img', id='main-image') or soup.select_one('img[src*="imagebam.com"]')

    if img and img.get('src'):
        return img['src']
    return None

def run_basics_scraper():
    if not os.path.exists("imagebam_links.txt"):
        print("Error: Need imagebam_links.txt first.")
        return

    with open("imagebam_links.txt", "r") as f:
        urls = [line.strip() for line in f.readlines()]

    session = requests.Session()
    # Inject the 'I have seen the warning' cookie manually
    session.cookies.set("nsfw_inter", "1", domain=".imagebam.com")

    print(f"Resolving {len(urls)} direct links...")

    for url in urls:
        direct_url = resolve_direct_link(session, url)

        if direct_url:
            print(f"Found: {direct_url}")
            # Download
            img_data = session.get(direct_url, headers={"Referer": url}).content
            filename = f"downloads/{url.split('/')[-1]}.jpg"
            with open(filename, 'wb') as f:
                f.write(img_data)
        else:
            print(f"Could not resolve: {url}")

        time.sleep(1) # Small delay to stay under the radar

run_basics_scraper()

Resolving 100 direct links...
Found: https://images3.imagebam.com/ba/c5/71/93d2ed147366911.jpg
Found: https://images3.imagebam.com/12/1d/df/5cddb2147367365.jpg
Found: https://images3.imagebam.com/55/77/b7/fc9b54147367005.jpg
Found: https://images3.imagebam.com/e7/0b/f9/bf21b3147367319.jpg
Found: https://images3.imagebam.com/e2/af/97/792f41147367718.jpg
Found: https://images3.imagebam.com/58/ab/2a/006799147367206.jpg
Found: https://images3.imagebam.com/24/eb/22/9ccacd147367059.jpg
Found: https://images3.imagebam.com/fd/1f/ca/0dbd68147367667.jpg
Found: https://images3.imagebam.com/d8/99/9b/0212da147367556.jpg
Found: https://images3.imagebam.com/a1/84/a7/03d63b147367126.jpg
Found: https://images3.imagebam.com/38/77/c0/9ccf4b147367736.jpg
Found: https://images3.imagebam.com/07/c1/d4/3ec751147366961.jpg
Found: https://images3.imagebam.com/53/e0/8b/2da8f6147367541.jpg
Found: https://images3.imagebam.com/0a/43/9b/cb4f95147367458.jpg
Found: https://images3.imagebam.com/42/c6/8a/cfc5ae147367625

In [9]:
import os

# 1. Create the Zip file
!zip -r /content/my_images.zip /content/downloads

# 2. Provide a download link (useful for mobile browsers)
from google.colab import files
files.download('/content/my_images.zip')

  adding: content/downloads/ (stored 0%)
  adding: content/downloads/cb4f95147367458.jpg (deflated 20%)
  adding: content/downloads/bf21b3147367319.jpg (deflated 10%)
  adding: content/downloads/5d7821147367232.jpg (deflated 7%)
  adding: content/downloads/d38bc7147367632.jpg (deflated 7%)
  adding: content/downloads/792f41147367718.jpg (deflated 4%)
  adding: content/downloads/5cddb2147367365.jpg (deflated 6%)
  adding: content/downloads/0750bb147367038.jpg (deflated 8%)
  adding: content/downloads/0b0157147367414.jpg (deflated 20%)
  adding: content/downloads/e0722b147367046.jpg (deflated 8%)
  adding: content/downloads/445aed147366953.jpg (deflated 21%)
  adding: content/downloads/0d16e6147367770.jpg (deflated 8%)
  adding: content/downloads/cfc5ae147367625.jpg (deflated 5%)
  adding: content/downloads/05051c147367154.jpg (deflated 8%)
  adding: content/downloads/ba6a34147366901.jpg (deflated 20%)
  adding: content/downloads/29ff40147367136.jpg (deflated 21%)
  adding: content/downl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>